In [1]:
import tensorflow_data_validation as tfdv
import pandas as pd
import datetime

from tensorflow_data_validation.utils import slicing_util
from typing import List, Optional, Text, Union, Dict, Iterable, Mapping


In [4]:
tfdv.__version__

'0.22.0'

In [2]:
data_location= 'gs://workshop-datasets/covertype/small/dataset.csv'
output_location = 'gs://mlops-dev-env/testing/tfdv-output'

In [3]:
!gsutil cat -r 0-500 {data_location}

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
3142,183,9,648,101,757,223,247,157,1871,Commanche,C7757,1
2156,18,28,0,0,1207,187,170,107,960,Cache,C6102,3
1967,124,16,60,9,124,245,227,105,451,Cache,C2704,2
3237,305,15,663,19,3593,178,231,193,1260,Commanche,C7201,0
2981,221,18,150,14,4584,195,254,191,1822,Rawah,C7

In [5]:
stats = tfdv.generate_statistics_from_csv(data_location)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [12]:
stats.datasets[0].features[0].path

step: "Wilderness_Area"

In [13]:
path = tfdv.FeaturePath("Wilderness_Area")

In [14]:
tfdv

In [38]:
df = pd.read_csv(data_location)
df

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3142,183,9,648,101,757,223,247,157,1871,Commanche,C7757,1
1,2156,18,28,0,0,1207,187,170,107,960,Cache,C6102,3
2,1967,124,16,60,9,124,245,227,105,451,Cache,C2704,2
3,3237,305,15,663,19,3593,178,231,193,1260,Commanche,C7201,0
4,2981,221,18,150,14,4584,195,254,191,1822,Rawah,C7745,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2710,245,10,175,41,417,200,248,185,335,Rawah,C7746,1
99996,2854,99,9,60,12,2581,235,228,123,2160,Rawah,C7745,1
99997,2936,64,7,285,23,4808,227,226,133,2804,Rawah,C7745,0
99998,2602,98,7,379,74,691,232,230,130,1445,Commanche,C2703,5


In [39]:
now = datetime.datetime.now() 

#timestamps = [(now + datetime.timedelta(minutes=60*(i//60)), now + datetime.timedelta(minutes=60*(i//60)+60)) for i in range(df.shape[0])]
#timestamps = [date1.isoformat(timespec='minutes') + '_' + date2.isoformat(timespec='minutes') for date1, date2 in timestamps]
time_window = ['C' + str((now + datetime.timedelta(seconds=i*4)).day) for i in range(df.shape[0])]

In [40]:
df['time_window'] = time_window
df['time_window'].unique()

array(['C16', 'C17', 'C18', 'C19', 'C20', 'C21'], dtype=object)

In [44]:
file_path = '/home/jarekk/workspace/test_dataset.csv'
df.to_csv(file_path, header=True, index=False)

In [43]:
!head {file_path}

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type,time_window
3142,183,9,648,101,757,223,247,157,1871,Commanche,C7757,1,C16
2156,18,28,0,0,1207,187,170,107,960,Cache,C6102,3,C16
1967,124,16,60,9,124,245,227,105,451,Cache,C2704,2,C16
3237,305,15,663,19,3593,178,231,193,1260,Commanche,C7201,0,C16
2981,221,18,150,14,4584,195,254,191,1822,Rawah,C7745,0,C16
2159,341,26,30,5,1340,157,192,168,1172,Cache,C6102,2,C16
3098,89,11,30,-6,1846,236,223,117,408,Rawah,C7202,0,C16
3089,24,9,285,18,3956,216,222,142,1956,Rawah,C7745,1,C16
2997,75,11,376,-3,3302,234,218,115,5642,Rawah,C7745,1,C16


In [51]:

slice_fn = slicing_util.get_feature_value_slicer(features={'time_window':None})

stats_options = tfdv.StatsOptions(
    slice_functions=[slice_fn]
)

In [52]:
file_path = '/home/jarekk/workspace/stats.pb'
file_path = '/home/jarekk/workspace/test_dataset.csv'
stats = tfdv.generate_statistics_from_csv(file_path, stats_options=stats_options)
#stats = tfdv.generate_statistics_from_dataframe(df, stats_options=stats_options)
#stats = tfdv.generate_statistics_from_dataframe(df)

In [53]:
tfdv.visualize_statistics(stats)

ValueError: lhs_statistics proto contains multiple datasets. Only one dataset is currently supported.

In [58]:
for dataset in stats.datasets:
    print('************************')
    print('************************')
    print(dataset)

 low_value: 123.0
        high_value: 147.60000000000002
        sample_count: 2003.7656000000002
      }
      buckets {
        low_value: 147.60000000000002
        high_value: 172.20000000000002
        sample_count: 1665.5455999999997
      }
      buckets {
        low_value: 172.20000000000002
        high_value: 196.8
        sample_count: 1101.8456
      }
      buckets {
        low_value: 196.8
        high_value: 221.4
        sample_count: 462.9856
      }
      buckets {
        low_value: 221.4
        high_value: 246.0
        sample_count: 102.21759999999996
      }
    }
    histograms {
      buckets {
        high_value: 94.0
        sample_count: 751.6
      }
      buckets {
        low_value: 94.0
        high_value: 112.0
        sample_count: 751.6
      }
      buckets {
        low_value: 112.0
        high_value: 124.0
        sample_count: 751.6
      }
      buckets {
        low_value: 124.0
        high_value: 134.0
        sample_count: 751.6
      }
  

In [57]:
stats

 avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 751.6
        